# How to use Google Gemini models with python
In this notebook we look into:
1. The basics on how to use a Google Gemini model with just a few lines of codes.
2. Which settings you can play with to tune the behaviour of the model on your use case.

## Google API setup

In order to use a Google Gemini models, you'll need to create an API key and configure it in your Google Colab Secrets.


1. You get your api key from Google AI Studio [here](https://aistudio.google.com/app/apikey)
4. Open your Colab secrets (click on the key icon here on the left)
3. Give a the name, for instance `GOOGLE_API_KEY`, and past the value in `Value`.
4. Toggle `Notebook access` to give access to this specific notebook to the API key.

🔑 Note that this api key will now be available in your secrets everytime you open or create a new colab notebook. You'll however still need to grant explicit access to each notebook.


💸 You'll be able to start **free of charge** but you will just be limited in the number of requests you could make to Gemini per minute and per day.
- I recommend you start using `gemini-2.0-flash` because you have 15 request per minute and a total of 1500 request per day free, so it's pretty good to strart.

In [7]:
# if you are running the notebook outside of Google Colab, uncomment this line to install the Google Generative AI library.
#!pip install -q -U google-generativeai

In [8]:
import google.generativeai as genai
import os

In [9]:
from google.colab import userdata
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')

In [10]:
genai.configure(api_key=GOOGLE_API_KEY)

## Simple inference with Gemini model from Google
Text generation is very simple.
- You need to create a `model` instance.
    - This is where you provide a `model_name` 🧠
- Then you call the `generate_content()` function.
    - THis is where you provide the user input query 💬


## Google Models
All the model we see are actually multimodals in that we could pass not just text but also Audio, Images, and Videos. But we focus on text for now.

I recommend testing models in the following order (from cheaper to more expensive and better).
1. `gemini-2.0-flash`: Fast and versatile performance across a diverse variety of tasks
    - Input token limit: **1 million !**
    - Knowledge Cutoff: Aug 2024.
    - 15 Request per minutes, 1500 requests per day on the free plan
2. `gemini-2.5-flash-preview-05-20` best model in terms of price-performance, offering well-rounded capabilities.
    - Input token limit: **1 million !**
    - Knowledge Cutoff: January 2025.
    - 10 Request per minutes, 500 requests per day on the free plan
3. `gemini-2.5-pro-preview-06-05`: One of the world's best model to-date. State-of-the-art thinking model, capable of reasoning over complex problems in code, math, and STEM, as well as analyzing large datasets, codebases, and documents using long context
    - Input token limit: **1 million !**
    - Latest update: January 2025.
    - 2 Request per minutes, 50 requests per day on the free plan

If you need more capacity you'll have to configure a billing account. And [here](https://ai.google.dev/pricing)


In [11]:
model = genai.GenerativeModel(model_name="gemini-2.0-flash")
response = model.generate_content("Write a very short poem about an astronaut on the Moon")
print(response.text)

Dust whispers secrets,
A silent, silver stage.
One small step echoes,
Across a cosmic age.



# Advanced Parameters

[here](https://ai.google.dev/gemini-api/docs/models/generative-models#model-parameters) is the documentation if you want to dig deeper, but I'll show you the most important ones below.

**Having a conversation**


In [12]:
# How to make an interactive chat?
model = genai.GenerativeModel("gemini-2.0-flash")
chat = model.start_chat(
    history=[
        {"role": "user", "parts": "Hello"},
        {"role": "model", "parts": "Great to meet you. What would you like to know?"},
    ]
)
response = chat.send_message("I have 2 dogs in my house.")
print(f"Model response 1:\n")
print(response.text)
print(f"Model response 2:\n")
response = chat.send_message("How many paws are in my house?")
print(response.text)

Model response 1:

That's wonderful! What kind of dogs are they? I'd love to hear more about them. Are they the same breed? Are they getting along well?

Model response 2:

Since you have two dogs, and each dog has four paws, there are 2 * 4 = 8 paws in your house!



**Configure text generation parameters**

Every prompt you send to the model includes [parameters](https://ai.google.dev/gemini-api/docs/models/generative-models#model-parameters) that control how the model generates responses. You can use [GenerationConfig](https://ai.google.dev/api/rest/v1/GenerationConfig) to configure these parameters. If you don't configure the parameters, the model uses default options, which can vary by model.

In [13]:
model = genai.GenerativeModel("gemini-2.0-flash")
response = model.generate_content(
    "Tell me a story about a magic backpack.",
    generation_config=genai.types.GenerationConfig(
        candidate_count=1, #  Number of generated responses to return
        stop_sequences=["x"], # step of character which stop generation. Rarely used.
        max_output_tokens=20, # The maximum number of tokens to include in a response candidate.
        temperature=1.0, # Controls the randomness of the output: Betweem 0.0 and 2.0
        presence_penalty=1.0,
        top_k=10
    ),
)

print(response.text)

Flora was a collector of oddities. Not e


**Note**: Like with OpenAI API you can control the following other parameters (for the full details look at [GenerationConfig](https://ai.google.dev/api/rest/v1/GenerationConfig)).
- response_schema;
- top_p
- top_k
- presence_penalty
- frequence_penalty
- logprobs
- ...

In [14]:
# You can also count tokens easily
print(response.usage_metadata)

prompt_token_count: 9
candidates_token_count: 10
total_token_count: 19



Every prompt you send to the model includes parameters that control how the model generates responses. You can use GenerationConfig to configure these parameters. If you don't configure the parameters, the model uses default options, which can vary by model.

# Streaming

In [15]:
model = genai.GenerativeModel("gemini-2.0-flash")
response = model.generate_content("Write a story about a magic backpack.", stream=True)
for chunk in response:
    print(chunk.text)
    print("_" * 80)

El
________________________________________________________________________________
ara was,
________________________________________________________________________________
 to put it mildly, uninspired. High school felt like a monotonous cycle of textbooks and
________________________________________________________________________________
 cafeteria lunches. She longed for something, anything, to disrupt the predictable rhythm of her life
________________________________________________________________________________
. Then, one rainy Tuesday, rummaging through her grandmother's dusty attic, she found it.

Tucked away in a cobweb-laden
________________________________________________________________________________
 corner was a backpack, not just any backpack, but one crafted from a deep indigo fabric that seemed to shimmer with an inner light. The leather straps were intricately
________________________________________________________________________________
 woven, and a single,